In [3]:
from funcions_finals.matricula_to_digits_2 import matricula_to_digits
from funcions_finals.digits_to_predicts_3 import predict_digits, CNNModel_a, CNNModel_n
import cv2
import numpy as np
import os
from matplotlib import pyplot as plt
import torch
import torch.nn as nn
import torch.nn.functional as F
from ultralytics import YOLO
from funcions_finals.detector_matricula import init_models
from funcions_finals.detector_matricula import detect_matricula
from funcions_finals.matricula_to_digits_2 import matricula_to_digits
from funcions_finals.digits_to_predicts_3 import predict_digits
from funcions_finals.image_to_matricula_1 import crop_matricula
import pandas as pd



In [4]:
# get data
final_df_path_img = 'dataset_final/Dataset-Final'
final_df_path_txt = 'dataset_final/Matriculas.xlsx'
final_df = pd.read_excel(final_df_path_txt,header=None)

# init models
model_yolo, model_alfa, model_num = init_models("models/CNN1-numeros.pt","models/CNN1-alfa.pt")

YOLOv5  2024-10-5 Python-3.9.7 torch-2.4.1+cpu CPU

Fusing layers... 
Model summary: 157 layers, 7012822 parameters, 0 gradients, 15.8 GFLOPs
Adding AutoShape... 


## 1. CROP MATRICULES ADQUIRIDES

In [15]:
total = 0
correct = []
incorrect = []
mat_preds = []
# Iterar sobre contenido de final_df_path_img
for row,img in enumerate(os.listdir(final_df_path_img)):
    total += 1
    # 1. get img path
    image_path = os.path.join(final_df_path_img, img)

    # 2. crop matricula
    matricula = crop_matricula(image_path, model_yolo)

    # 3. save in folder tmp_cropped as tmp_cropped_{row}.jpg
    matricula_path = 'tmp_cropped_2'
    matricula_path = os.path.join(matricula_path, f'tmp_cropped_{row}.jpg')
    cv2.imwrite(matricula_path, matricula)


c:\Users\mirvi\Desktop\mii\UAB\4.1\PSIV2\detect mateicules\r1\yolov5\models\common.py:892: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with amp.autocast(autocast):
c:\Users\mirvi\Desktop\mii\UAB\4.1\PSIV2\detect mateicules\r1\yolov5\models\common.py:892: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with amp.autocast(autocast):
c:\Users\mirvi\Desktop\mii\UAB\4.1\PSIV2\detect mateicules\r1\yolov5\models\common.py:892: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with amp.autocast(autocast):
c:\Users\mirvi\Desktop\mii\UAB\4.1\PSIV2\detect mateicules\r1\yolov5\models\common.py:892: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with amp.autocast(autocast):
c:\Users\mirvi\Deskt

## 2. VALIDAR FIND CONTOURS

In [10]:
possible_errors = []
correct = []
error = 0
total = 0
for i,img in enumerate(os.listdir("tmp_cropped_2")):
    # 1. get img path
    image_path = os.path.join("tmp_cropped_2", img)
    
    # 2. matricula to digits
    digits = matricula_to_digits(image_path)

    if len(digits) != 7:
        possible_errors.append([image_path, digits])
        error +=1
    else:
        correct.append((image_path, digits))
    
    total += 1

print('ERRORS:',error)

ERRORS: 0


### 2.1 MOSTREM ERRORS

In [6]:
for err in possible_errors:
    print(err[0])

### 2.2 DESEM DIGITS TROBATS

In [9]:
df_num = 'digits_reals/num'
df_alfa = 'digits_reals/alfa'
# Iterar sobre las imágenes
i=0
x = 0
for img, dg in correct:
    j=0
    for d in dg:
        if j<3:
            # Generar la ruta del archivo
            digits_path = os.path.join(df_alfa, f'digits_{x}.jpg')
        else:
            digits_path = os.path.join(df_num, f'digits_{x}.jpg')

        # Guardar la imagen en la carpeta especificada
        cv2.imwrite(digits_path, d)
        j+=1
        x+=1
    
    i+=1

La validació de IMG CLASSIF està feta en un arxiu apart, ja que implicava més feina per classificar a mà cadascun dels digits


## 3. MOSTREM RESULTATS FINALS

In [1]:
from funcions_finals.matricula_to_digits_2 import matricula_to_digits
from funcions_finals.digits_to_predicts_3 import predict_digits, CNNModel_a, CNNModel_n
import cv2
import numpy as np
import os
from matplotlib import pyplot as plt
import torch
import torch.nn as nn
import torch.nn.functional as F
from ultralytics import YOLO
from funcions_finals.detector_matricula import init_models
from funcions_finals.detector_matricula import detect_matricula
from funcions_finals.matricula_to_digits_2 import matricula_to_digits
from funcions_finals.digits_to_predicts_3 import predict_digits
from funcions_finals.image_to_matricula_1 import crop_matricula
import pandas as pd

In [2]:
def metric_validator(str_pred, str_gt):
    punt = 0
    for i in range(len(str_pred)):
        if str_pred[i] == str_gt[i]:
            punt += 1
    return punt

In [3]:
final_df_path_img = 'dataset_final/Dataset-Final'
final_df_path_txt = 'dataset_final/Matriculas.xlsx'
final_df = pd.read_excel(final_df_path_txt,header=None)

model_yolo, model_alfa, model_num = init_models("models/CNN_num_rsm.pt","models/CNN_alfa_rsm.pt")

punts_total = 0
punts_max = 0
for row,img in enumerate(os.listdir(final_df_path_img)):
    # 1. get img path
    image_path = os.path.join(final_df_path_img, img)
    image_number = int(img.split('.')[0])

    # 2. crop matricula
    matricula = crop_matricula(image_path, model_yolo)

    # 3. get digitsS
    digits = matricula_to_digits(matricula)

    # 4. get predictions
    numeric_preds, alfa_preds, full_str = predict_digits(digits, model_num, model_alfa)

    # 5. comparem amb el ground truth
    ground_truth = final_df.iloc[image_number-1, 0]
    punts_total += metric_validator(full_str, ground_truth)
    punts_max += 7



YOLOv5  2024-10-5 Python-3.9.7 torch-2.4.1+cpu CPU

Fusing layers... 
Model summary: 157 layers, 7012822 parameters, 0 gradients, 15.8 GFLOPs
Adding AutoShape... 
c:\Users\mirvi\Desktop\mii\UAB\4.1\PSIV2\detect mateicules\r1\yolov5\models\common.py:892: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with amp.autocast(autocast):
c:\Users\mirvi\Desktop\mii\UAB\4.1\PSIV2\detect mateicules\r1\yolov5\models\common.py:892: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with amp.autocast(autocast):
c:\Users\mirvi\Desktop\mii\UAB\4.1\PSIV2\detect mateicules\r1\yolov5\models\common.py:892: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with amp.autocast(autocast):
c:\Users\mirvi\Desktop\mii\UAB\4.1\PSIV2\detect mateicules\r1\yolov5\models\common.py:892: FutureWarni

In [4]:
# Correct letters / Total Letters
print('Accuracy:', punts_total/punts_max*100, '%')
print(punts_total, '/', punts_max)

Accuracy: 50.31055900621118 %
81 / 161


In [13]:
from funcions_finals.digits_to_predicts_3 import predecir_imagen

dig_path = "digits_reals/alfa/F_1.jpg"

model_yolo, model_alfa, model_num = init_models("models/CNN5-num.pt","models/CNN5-alfa.pt")

img = cv2.imread(dig_path, cv2.IMREAD_GRAYSCALE)

pred = predecir_imagen(model_alfa,img)

predicted_letter = chr(pred + ord('A'))

print(predicted_letter)

YOLOv5  2024-10-5 Python-3.9.7 torch-2.4.1+cpu CPU

Fusing layers... 
Model summary: 157 layers, 7012822 parameters, 0 gradients, 15.8 GFLOPs
Adding AutoShape... 


F


## PROVES INDIVIDUALS FINALS


In [13]:
model_yolo, model_alfa, model_num = init_models("models/CNNog-num2.pt","models/CNN1-alfa2.pt")

# IMG PATH
image_path = 'dataset_final/Dataset-Final/4.jpg'

# 1. crop matricula
matricula = crop_matricula(image_path, model_yolo)

# 2. get digits
digits = matricula_to_digits(matricula)

# 3. get predictions
numeric_preds, alfa_preds, full_str = predict_digits(digits, model_num, model_alfa)
print(full_str)


YOLOv5  2024-10-5 Python-3.9.7 torch-2.4.1+cpu CPU

Fusing layers... 
Model summary: 157 layers, 7012822 parameters, 0 gradients, 15.8 GFLOPs
Adding AutoShape... 


FileNotFoundError: [Errno 2] No such file or directory: 'models/CNN1-alfa2.pt'

5796DKP
